In [1]:
## For debbugging
%load_ext autoreload

In [2]:
## for debugging:
%aimport mlp.keyword
%aimport mlp.tools
%autoreload 1
from mlp.keyword import *

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline


In [5]:
# data_directory = os.path.join('.', 'tutorial_data')

nl_col = {
    "Description": 5,
    "Resolution": 6
#     "Resolution": 248
}

# meta_col = {
#     "AssetID": 6,
#     "Location": 7,
#     "Assigned": 10,
#     "Reported": 43,
#     "SolveDate": 11,
#     "ReportDate": 44
# }

kex = KeywordExtractor('Maintenance_All_Clean.xlsx',
                      nlp_cols=nl_col,
                      special_replace={' -':'; '})

# kex = KeywordExtractor('hvac_mslips_20170720.xlsx',
#                       nlp_cols=nl_col,
#                       meta_cols = meta_col,
#                       special_replace={'Action Taken:': ''},
#                       wdir=data_directory)

{' -': '; '}


In [13]:
kex.fit(vocab='tag_vocab.csv')
# kex.fit(vocab='hvac_vocab.csv')

In [14]:
kex.vocab.head()

,NE,alias,note
token,,,
replace,S,replace,NaN
unit,I,unit,NaN
motor,I,motor,NaN
spindle,I,spindle,NaN
leak,P,leak,NaN


In [15]:
# kex.gen_vocab('testing_vocab.xlsx')

In [16]:
# import yappi

In [18]:
# yappi.start()

df_pred = kex.transform()

# yappi.get_func_stats().print_all()

100%|██████████| 3430/3430 [00:06<00:00, 514.56it/s]


In [20]:
df_pred[df_pred.UK_tok ==''].shape[0]

403

In [21]:
print(df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].shape[0])
df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)]

61


,RawText,Items,Problem,Solution,UK_tok
609,accumulator_check ok at_30bar,,,,"at_30bar, accumulator_check"
630,unload automation return,,,,"automation, unload"
709,bar_end find production pan,,,,"bar_end, bar_end find, pan, find"
876,part_catcher will_not return in_auto,,,,"in_auto, will_not, part_catcher"
923,base_clean req base_clean,,,,"base_clean req, base_clean, base_clean req bas..."
924,base_clean req base_clean,,,,"base_clean req, base_clean, base_clean req bas..."
1081,base_cleaning request_complete,,,,"base_clean request_complete, request_complete,..."
1084,base_cleaning request_complete,,,,"base_clean request_complete, request_complete,..."
1136,accumulator_check no_issue,,,,"no_issue, accumulator_check, accumulator_check..."
1152,base_cleaning request_complete,,,,"base_clean request_complete, request_complete,..."
